In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import boto3

In [ ]:
s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

In [ ]:
s3.Bucket("digpath-data")